## References
### Wikipedia
[Iterated Function Systems](https://en.wikipedia.org/wiki/Iterated_function_system)

[Chaos Game](https://en.wikipedia.org/wiki/Chaos_game)

[L-systen](https://en.wikipedia.org/wiki/L-system)

[Barnsley Fern](https://en.wikipedia.org/wiki/Barnsley_fern)

### Python Packages

[turtle graphics](https://docs.python.org/3/library/turtle.html)<br>
Python 3.10 lib/turtle.py documentation

[The Chaos Game](https://beltoforion.de/en/recreational_mathematics/chaos_game.php)<br>
An implementation of the Chaos Game using polygons

## Mathematica
[SubstitutionSystem](https://reference.wolfram.com/language/ref/SubstitutionSystem.html)

In [1]:
from music21 import stream, interval, corpus, instrument, clef, key
from music21 import converter, note, chord, environment, duration
from music21.stream import Score, Part, Measure
import notebook
import argparse
import pandas as pd
import pathlib
import numpy as np

from turtle import *
import random as rand
import math, sys, copy, re
from itertools import *
from operator import *

verbose = 0

In [2]:
from music21 import stream, interval, corpus, instrument, clef, key
from music21 import converter, note, chord, environment, duration, pitch
import pandas as pd

class MusicScale(object):
    def __init__(self,  resource_folder ="/Compile/dwbzen/resources/music", \
                 scale_name='Major', root_note=note.Note('C4'), key=key.Key('C')):
        
        self.scales_df = pd.read_json(resource_folder + "/commonScaleFormulas.json", orient='records').transpose()
        self.scale = self.scales_df.loc[scale_name]
        self.root = root_note
        self.scale_name = scale_name
        self.formula = self.scale['formula']
        self.key = key
        self.notes_stream = None
        self.scale_notes = self.get_scale_notes(root_note)
        self.scale_notes_names = [x.nameWithOctave for x in self.scale_notes]
        self.range_notes = self.get_range_notes(root_note)
        self.range_notes_names = [x.nameWithOctave for x in self.range_notes]
        
    def get_scale_notes(self, start_note=None):
        '''The notes of the configured scale spanning a single octave, for example 'C4' to 'C5'
            Note that the top note is the same as the bottom note but an octave higher.
            Returns: a [Note]
        '''
        ns = stream.Stream()
        ns.append(self.key)
        if start_note is None:
            n = self.root
        else:
            n = start_note
        ns.append(n)
        for i in range(len(self.formula)):
            n = n.transpose(self.formula[i], inPlace=False)
            ns.append(n)
        return [x for x in ns.notes]
    
    def get_range_notes(self, start_note=None, start_octave=0, end_octave=8):
        '''For example, with default arguments notes (Major scale) are C0,D0,...B7,C8
        
        '''
        if start_note is None:
            n = self.root
            start_note = self.root
        else:
            n = start_note
        notes_list = []
        for octave in range(start_octave, end_octave):
            for n in self.scale_notes[:-1]:
                new_note = note.Note(n.name+str(octave))
                notes_list.append(new_note)
        new_note = note.Note(start_note.name+str(end_octave))
        notes_list.append(new_note)
        return notes_list
    
    def get_note(self, scale_degrees:int=0, from_note=None) -> note.Note:
        '''Gets a new Note that is the given scale_degrees away from a given Note.
            Arguments:
                scale_degrees - any integer, the sign dictates the direction - up or down
                from_note - the starting note, default is the scale root note
            Returns: a new Note
            Note if from_note is not present in the full range of notes (self.range_notes), the from_note is returned.
        '''
        if from_note is None:
            from_note = self.root

        if from_note.nameWithOctave in self.range_notes_names:
            ind = self.range_notes_names.index(from_note.nameWithOctave) + scale_degrees
            return self.range_notes[ind]
        else:
            return from_note


In [3]:
mscale = MusicScale()
print(mscale.scale_notes)
print(mscale.formula)
n = mscale.scale_notes[2]
print(n.nameWithOctave)
rn = mscale.get_range_notes(start_octave=4, end_octave=6)
print([x.nameWithOctave for x in rn])

rn = mscale.get_range_notes()
e4 = rn[30]
anote = note.Note("E4", quarterLength=2)
mscale.get_note(3, anote)

[<music21.note.Note C>, <music21.note.Note D>, <music21.note.Note E>, <music21.note.Note F>, <music21.note.Note G>, <music21.note.Note A>, <music21.note.Note B>, <music21.note.Note C>]
[2, 2, 1, 2, 2, 2, 1]
E4
['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'D5', 'E5', 'F5', 'G5', 'A5', 'B5', 'C6']


<music21.note.Note A>

## Combined Rules

The previous code has two  rule sets for pitch and duration that were applied independently: the pitch rules first, then the duration rules on the result.  This isn't quite what I had envisioned.

This version combines pitch and duration substitution into a single rule. The downside is the rules are more complex and can no longer be represented by a 2 characters (pitch) or number (duration).<br>
However it is more intuitive and easier to construct logically. For example a rule might be<br>
**replace a given note with 4 notes with pitches 0, +1, -2, +1, and duration half the existing duration**

The pitch rule is expressed as an interval from the previous pitch as before.<br>
The duration rule is expressed as a multiple of the duration of the previous note.

In addition each substitution has optional pre- and post- processing rule(s). The duration post-processing rule for example
would check if the resulting duration is less than, say, a 16th note (0.25 quarterLength) and if so set to 1 quarterLength.<br>
The RuleSet class then will consist of the following:
1. Pre-processing rules expressed as functions.
2. Pitch, duration substitutions as an ordered pair: (pitch, duration). For example ('0+1-2+1', 0.5)
3. Post-processing rules expressed as function(s).

Rule application is recursive: each new generation is created by applying the rule set to the previous generation.

In [4]:
from music21 import stream, interval, corpus, instrument, clef, key, meter
from music21 import converter, note, chord, environment, duration, tempo, metadata
from music21.stream import Score, Part, Measure
import pandas as pd
import sys, copy, re

trace_processing = False

def duration_rule(cmd, anote):
    '''Apply the duration command cmd to Note anote.
        Arguments:
            cmd - a float to multiply the Notes duration.quarterLength by
            anote - a Note
    '''
    if trace_processing:
        print(f" apply duration({cmd})")
    old_dur = anote.duration.quarterLength
    new_dur = round(old_dur * cmd, 3)
    new_note = note.Note(anote.nameWithOctave, quarterLength=new_dur)
    if trace_processing:
        print(f"old_dur: {old_dur}  new_dur: {new_dur}")
    return new_note

def duration_postprocess(anote:note.Note, instrument_name:str):
    if trace_processing:
        print('duration_postprocess')
    current_dur = anote.duration.quarterLength
    if current_dur > 4.0:
        new_dur = current_dur/2
    elif current_dur < 0.125:
        new_dur = current_dur * 4
    new_note = note.Note(anote.nameWithOctave, quarterLength=new_dur)
    return new_note

def pitch_postprocess(anote:note.Note, instrument_name:str):
    '''Checks the pitch against the range of the given instrument, adjusting if necessary.
    '''
    if trace_processing:
        print('pitch_postprocess')


def score_preprocess(ascore:Score):
    if trace_processing:
        print('score_preprocess')


def interval_rule(x, state):
    if trace_processing:
        print(f" apply interval({x})")
    scale_degrees = int(x) * state['direction']
    last_note = state['note']
    last_note_dur = last_note.duration.quarterLength
    scale_note = state['musicScale'].get_note(scale_degrees, last_note)
    next_note = copy.deepcopy(scale_note)
    next_note.duration = duration.Duration(last_note_dur)
    #
    #
    state['note'] = next_note
    state['part'].append(next_note)
    if trace_processing:
        show('scale_note', scale_note)
        show('next_note', next_note)
    return next_note



In [5]:
import re

class RuleSet(object):

    def __init__(self, substitution_rules:dict, rules=None):
        self.rules = rules
        self.substitutions = []
        self.substitution_rules = substitution_rules
        #
        # Each substitution is a dict. The key is a regular expression string:
        # (interval)/(quarterLength-multiplier) used to match an input string.
        # The value is the replacement. The re groups are named and can be
        # referenced in the replacement.
        # Sample substitutions:
        #   sub1 = {r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}
        #   sub2 = {r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/<duration>', '1/0.5']}
        
        #
        # compile the substitution rules regular expressions
        #
        for k in self.substitution_rules.keys():
            pattern = re.compile(k)
            replacement = self.substitution_rules[k]
            self.substitutions.append({'pattern':pattern, 'replacement':replacement})

        if rules is not None:
                self.pre_processing = rules['preProcessing'] 
                self.post_processing = rules['postProcessing']

In [6]:
#
# test RuleSet
#
substitution_rules = {\
        r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0'], \
        r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/<duration>', '1/0.5']\
}
ruleSet = RuleSet(substitution_rules)
print(f'substitutions: {ruleSet.substitutions}\n')
for subs in ruleSet.substitutions:
    print(f'sub: {subs}')
    print(f' pattern: {subs["pattern"]}\n replacement: {subs["replacement"]}\n' )

substitutions: [{'pattern': re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}, {'pattern': re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/<duration>', '1/0.5']}]

sub: {'pattern': re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}
 pattern: re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)')
 replacement: ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']

sub: {'pattern': re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/<duration>', '1/0.5']}
 pattern: re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)')
 replacement: ['0/<duration>', '1/0.5']



In [23]:
import re

class SubstitutionSystem(object):
    '''Generates a list representing the evolution of the substitution system with the specified rule 
        from initial condition init for t steps.
        Arguments:
            substitution_rules - 
            start - initial condition
            t - the number of steps (generations) as a range(n1, n2)
                where n1 is the starting step, and n2-1 is the ending step, 0 <= n1 <= n2
                For example, t=range(1, 4) will output the results of steps 1,2 and 3
                t=range(2, 3) step 2 only.
                t=range(2, 4) steps 2 and 3
    '''
    
    def __init__(self, rule_set:RuleSet, verbose=0):
        self.rule_set = rule_set
        self.substitutions = rule_set.substitutions
        self.verbose = verbose
    
    def apply(self, start:[str], nsteps):
        sub_result = start
        self.steps = nsteps
        if nsteps > 0:
            sub_result = start
            for step in range(nsteps):
                sub_result = self.apply_step(sub_result)
                if self.verbose > 0:
                    print(f'step: {step}\nresult: {sub_result}')
        return sub_result
            
        
    def apply_step(self, start:[str]):
        subst_result = []
        for subs in self.substitutions:
            pattern = subs["pattern"]
            replacement = subs["replacement"]

            # try to match each item in start (result) list
            # 
            for s in start:
                if self.verbose > 0:
                    print(f's: "{s}"    pattern: {pattern.pattern}')
                match = pattern.match(s)
                if match is not None:
                    if self.verbose > 0:
                        print(f'"{s}" matched pattern {pattern.pattern} ')
                    grp_dict = match.groupdict()
                    #
                    # now substitute
                    # TODO - replace <fieldname> with the matched value in grp_dict in each replacement
                    #
                    subst_result += replacement
                else:
                    subst_result += [s]
            if self.verbose > 0:
                print(f'step result: {subst_result} \n')
        return subst_result
                        

In [24]:
start = ['0/1.0', '+1/0.5']
substitution_rules = {\
        r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0'], \
        r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/1.0', '1/0.5']\
}
ruleSet = RuleSet(substitution_rules)
ss = SubstitutionSystem(ruleSet, verbose=1)

In [25]:
sub_result = ss.apply(start,2)
print(sub_result)

s: "0/1.0"    pattern: (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)
"0/1.0" matched pattern (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+) 
s: "+1/0.5"    pattern: (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)
step result: ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0', '+1/0.5'] 

s: "0/1.0"    pattern: (?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)
s: "+1/0.5"    pattern: (?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)
"+1/0.5" matched pattern (?P<interval>[+-]?1)/(?P<duration>\d+\.\d+) 
step result: ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0', '+1/0.5', '0/1.0', '0/1.0', '1/0.5'] 

step: 0
result: ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0', '+1/0.5', '0/1.0', '0/1.0', '1/0.5']
s: "0/0.5"    pattern: (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)
"0/0.5" matched pattern (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+) 
s: "+1/1.0"    pattern: (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)
s: "-2/1.0"    pattern: (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)
s: "+1/2.0"    pattern: (?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)
s

In [11]:
import re
# the generic match string
re_str = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
re = re.compile(re_str)
m = re.match('0/0.5')
print(m.groups())
m = re.match('+1/1.0')
print(m.groupdict()['duration'])

('0', '0.5')
1.0


In [13]:
# match string for specific rule
import re
rule1_re_str = r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)'
rule1_re = re.compile(rule1_re_str)   # re.Pattern
rule2_re_str = r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)'
rule2_re = re.compile(rule2_re_str)
m1 = rule1_re.match('0/0.5')
print(m1.groupdict())

m2 = rule2_re.match('+1/1.0')
if m2 is not None:
    print(m2.groupdict())

{'interval': '0', 'duration': '0.5'}
{'interval': '+1', 'duration': '1.0'}


In [31]:
# SubstitutionSystem[{"0" -> "0+1-2+1", "1" -> "0+1", "2" -> "0-2"}, "0+1", {1}]
#   {"0+1-2+1+0+1"}
#
# SubstitutionSystem[{"0" -> "0+1-2+1", "1" -> "0+1", "2" -> "0-2"}, "0+1", {2}]
#   {"0+1-2+1+0+1-0-2+0+1+0+1-2+1+0+1"}
#
start = ["0+1"]
substitution_rules = {\
    r'(?P<interval>0)': ['0', '+1', '-2', '+1'],
    r'(?P<interval>1)': ['0', '+1'],
    r'(?P<interval>2)': ['0', '-2']
}
ruleSet = RuleSet(substitution_rules)
ss = SubstitutionSystem(ruleSet, verbose=0)
ss.apply(start, 1)

['0', '+1', '-2', '+1', '0+1', '0+1']